In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV, cross_validate
import time
from datetime import timedelta

In [4]:
df = pd.read_csv('../data/processed/train_small_no_sarcasm.csv')
random_state = 42

# Shuffle the DataFrame
df = df.sample(frac=1, random_state=random_state).reset_index(drop=True)
print(df.head())

# OCHO MI DA ERRORE PER DEI NAN
df = df.dropna(subset=['tweet'])

# Label encoding
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Check the mapping
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(label_mapping)

X_train = df['tweet']
y_train = df['label']

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)

                                               tweet     label
0                  looking forward meeting hope help  positive
1  buzzin win even draw like looking moment point...  negative
2  dancer novel hardcover acclaimed author side b...  negative
3  matter happens want travelling around europe a...  positive
4            take vodashop would asking one mxm next  negative
{'negative': 0, 'positive': 1}


In [24]:
models = [
    #LogisticRegression(fit_intercept=False, random_state=random_state),
    #LinearSVC(random_state=random_state),
    KNeighborsClassifier(n_jobs=-1),
    DecisionTreeClassifier(class_weight='balanced', random_state=random_state),
    MLPClassifier(verbose=False, random_state=random_state),
    RandomForestClassifier(class_weight='balanced', random_state=random_state),
    GradientBoostingClassifier(random_state=random_state),
    AdaBoostClassifier(random_state=random_state),
    ExtraTreesClassifier(class_weight='balanced', random_state=random_state),
    RidgeClassifier(class_weight='balanced', random_state=random_state),
    SGDClassifier(class_weight='balanced', random_state=random_state),
    GaussianNB(),
    RadiusNeighborsClassifier(weights='distance'),
    QuadraticDiscriminantAnalysis(),
    LinearDiscriminantAnalysis(),
    HistGradientBoostingClassifier(random_state=random_state),
    XGBClassifier(random_state=random_state, use_label_encoder=False, eval_metric='mlogloss'),
    LGBMClassifier(random_state=random_state)
]

          
models_names = [
    #'Logistic Regression',
    #'Support Vector Machine',
    'K Nearest Neighbors',
    'Decision Tree',
    'Multi Layer Perceptron',
    'Random Forest',
    'Gradient Boosting',
    'AdaBoost',
    'Extra Trees',
    'Ridge Classifier',
    'SGD Classifier',
    'Gaussian Naive Bayes',
    'Radius Neighbors',
    'Quadratic Discriminant Analysis',
    'Linear Discriminant Analysis',
    'Histogram-based Gradient Boosting'
]

models_hparams = [
    #{'solver': ['saga'], 'penalty': ['l2'], 'C': [9e-2], 'fit_intercept':[True]},
    
    #{'penalty': ['l2'], 'C': [3.75e-2], 'fit_intercept':[True]},
    
    {'n_neighbors': list(range(1, 10, 2)), 'weights':['uniform', 'distance'], 'leaf_size':[30, 50]},
    
    {'max_depth': [3, 4, 5, 7, 10], 'criterion': ['gini', 'entropy']},
    
    {'hidden_layer_sizes': [(40), (50), (80)], 'max_iter': [100, 200],
     'activation': ['logistic', 'relu'], 'solver': ['lbfgs', 'sgd', 'adam'], 'alpha': [1e-4, 1e-2, 1, 1e1, 1e2]},
    
    {'n_estimators': [50, 100, 200], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [None, 10, 20, 30], 'criterion': ['gini', 'entropy']},
    
    {'learning_rate': [0.01, 0.1, 0.2, 0.3], 'n_estimators': [50, 100, 200], 'max_depth': [3, 4, 5, 7, 10]},
    
    {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.5, 1]},
    
    {'n_estimators': [50, 100, 200], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [None, 10, 20, 30], 'criterion': ['gini', 'entropy']},
    
    {'alpha': [0.1, 1.0, 10.0], 'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg']},
    
    {'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge'], 'penalty': ['l2', 'l1', 'elasticnet'], 'alpha': [0.0001, 0.001, 0.01, 0.1]},
    
    {},  # GaussianNB has no hyperparameters to tune
    
    {'radius': [1.0, 1.5, 2.0, 2.5, 3.0], 'weights': ['uniform', 'distance']},
    
    {'reg_param': [0.0, 0.1, 0.5, 1.0], 'tol': [0.0001, 0.001, 0.01, 0.1]},
    
    {'solver': ['svd', 'lsqr', 'eigen'], 'shrinkage': ['auto', None, 0.1, 0.5, 1.0]},
    
    {'learning_rate': [0.01, 0.1, 0.2], 'max_iter': [100, 200], 'max_leaf_nodes': [31, 127, 255], 'max_depth': [None, 3, 5, 7, 10]}
]


In [25]:
chosen_hparams = list()
estimators = list()

for model, model_name, hparams in zip(models, models_names, models_hparams):
    
        print("\n########       {}       ########".format(model_name))
        starting_time = time.time()
        clf = GridSearchCV(estimator=model, param_grid=hparams, scoring='accuracy', cv=5)
        clf.fit(X_train, y_train)
        ending_time = time.time()
        chosen_hparams.append(clf.best_params_)
        estimators.append((model_name, clf.best_score_, clf.best_estimator_))
        
        for hparam in hparams:
            print(f'\t--> best value for hyperparameter "{hparam}": ', clf.best_params_.get(hparam))
            
        print(f'\t--> accuracy: ', clf.best_score_)
        print(f'\tElapsed time for GridSearch: ', timedelta(seconds=ending_time - starting_time))
        



########       K Nearest Neighbors       ########


In [ ]:
### Ensemble: Stacking Classifier with top 3 weak learners

# Sort estimators by the balanced accuracy metric
estimators.sort(key=lambda i:i[1],reverse=True)

# Get the top 3 classifiers by their accuracy metric
top3_clfs = list()
for clf in estimators[0:3]:
    top3_clfs.append((clf[0], clf[2]))
    
# Instantiate the Stacking Classifier with the top 3 weak learners
clf_stack = StackingClassifier(estimators = top3_clfs, final_estimator = LogisticRegression())

In [ ]:
perf_eval_estimators = list()
for model_tuple in estimators:
    model_name = model_tuple[0]
    model = model_tuple[2]
    scores = cross_validate(model, X_train, y_train, cv=5, scoring=('accuracy'))
    print('\n')
    print('The cross-validated Accuracy of {} is: '.format(model_name), np.mean(scores['test_score']))
    perf_eval_estimators.append((model_name, np.mean(scores['test_score']), model))

# Cross Validation for Stacking Ensemble
scores = cross_validate(clf_stack, X_train, y_train, cv=5, scoring=('accuracy'))
print('\n')
print('The cross-validated Accuracy of Stacking Model is ', np.mean(scores['test_score']))

perf_eval_estimators.append( ('Stacking Classifier', np.mean(scores['test_score']), clf_stack) )

perf_eval_estimators.sort(key=lambda i:i[1],reverse=True)
final_model = perf_eval_estimators[0][2]
final_model_accuracy = perf_eval_estimators[0][1]
final_model_name = perf_eval_estimators[0][0]
print("\n######## The Final Model selected is: ########")
print(final_model_name)
print('The cross-validated Accuracy is: ', final_model_accuracy)

In [ ]:
# Final training
final_model.fit(X_train, y_train)

In [ ]:
# Tested model hparams
models_hparams = [
    {'solver': ['liblinear', 'saga'], 'penalty': ['l1', 'l2'], 'C': [1e-5, 5e-5, 1e-4, 5e-4, 1], 'fit_intercept':[True, False]},
    
    {'C': [1e-4, 1e-2, 1, 1e1, 1e2], 'gamma': ['scale', 1e-2, 1e-3, 1e-4, 1e-5], 'kernel': ['linear', 'rbf']},
    
    {'n_neighbors': list(range(1, 10, 2))},
    
    {'max_depth': [3, 4, 5, 7, 10], 'criterion': ['gini', 'entropy']},
    
    {'hidden_layer_sizes': [(40), (50), (80)], 'max_iter': [100, 200],
     'activation': ['logistic', 'relu'], 'solver': ['lbfgs', 'sgd', 'adam'], 'alpha': [1e-4, 1e-2, 1, 1e1, 1e2]},
    
    {'n_estimators': [50, 100, 200], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [None, 10, 20, 30], 'criterion': ['gini', 'entropy']},
    
    {'learning_rate': [0.01, 0.1, 0.2, 0.3], 'n_estimators': [50, 100, 200], 'max_depth': [3, 4, 5, 7, 10]},
    
    {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.5, 1]},
    
    {'n_estimators': [50, 100, 200], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [None, 10, 20, 30], 'criterion': ['gini', 'entropy']},
    
    {'alpha': [0.1, 1.0, 10.0], 'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg']},
    
    {'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge'], 'penalty': ['l2', 'l1', 'elasticnet'], 'alpha': [0.0001, 0.001, 0.01, 0.1]},
    
    {},  # GaussianNB has no hyperparameters to tune
    
    {'radius': [1.0, 1.5, 2.0, 2.5, 3.0], 'weights': ['uniform', 'distance']},
    
    {'reg_param': [0.0, 0.1, 0.5, 1.0], 'tol': [0.0001, 0.001, 0.01, 0.1]},
    
    {'solver': ['svd', 'lsqr', 'eigen'], 'shrinkage': ['auto', None, 0.1, 0.5, 1.0]},
    
    {'learning_rate': [0.01, 0.1, 0.2], 'max_iter': [100, 200], 'max_leaf_nodes': [31, 127, 255], 'max_depth': [None, 3, 5, 7, 10]}
]